<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [1]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd

Reloading net


<h3>Set parameters</h3>

In [2]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 256

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

load_partial_net = True
CURR_EPOCH = 0

<h3>Import dataset</h3>

In [3]:
imageSet = ImageSet();
dataset = "horse2zebra"
imageSet.loadData(dataset, 'train', im_size)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/horse2zebra
Finished loading data


<h3>Define the CycleGAN network</h3>

In [4]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [5]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [6]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [7]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/"):
    os.mkdir("./img/")
    
if not os.path.isdir("./log/"):
    os.mkdir("./log/")
    
if os.path.isfile("./log/loss.log") and not load_partial_net:    
    os.remove("./log/loss.log")
    
logfile = open("./log/loss.log","a")

<h3> Load Partial Net </h3>

In [8]:
if (load_partial_net):
    if os.path.isfile("./model/model_info.txt"):
        model_file = open("./model/model_info.txt", "r")
        CURR_EPOCH = int(model_file.read())
        model_file.close()
    G.load_state_dict(torch.load('./model/G.data'))
    F.load_state_dict(torch.load('./model/F.data'))
    Dx.load_state_dict(torch.load('./model/Dx.data'))
    Dy.load_state_dict(torch.load('./model/Dy.data'))
    gan_opt.load_state_dict(torch.load('./model/gan_opt.data'))
    dx_opt.load_state_dict(torch.load('./model/dx_opt.data'))
    dy_opt.load_state_dict(torch.load('./model/dy_opt.data'))
    
last_epoch = -1
if (load_partial_net):
    last_epoch = CURR_EPOCH-1

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)

<h3> Training </h3>

In [ ]:
start_time = datetime.datetime.now()
for i in range(CURR_EPOCH,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x)
        dX_Y = Dx(mapX_Y)
        lossX_Y = gan_loss(dX_Y, dReal)
        
        mapY_X = F(src_y)
        dY_X = Dy(mapY_X)
        lossY_X = gan_loss(dY_X, dReal)
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    plt.imshow(np.transpose(torch.squeeze(mapX_Y.data.cpu()).numpy(), (1,2,0)), interpolation='nearest')
    tv.save_image(src_x.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    

    if not os.path.isdir("./model/"):
        os.mkdir("./model/")
    model_file = open("./model/model_info.txt", "w")
    model_file.write("%d"%(i+1))
    model_file.close()
    torch.save(G.state_dict(), './model/G.data')
    torch.save(F.state_dict(), './model/F.data')
    torch.save(Dx.state_dict(), './model/Dx.data')
    torch.save(Dy.state_dict(), './model/Dy.data')
    torch.save(gan_opt.state_dict(), './model/gan_opt.data')
    torch.save(dx_opt.state_dict(), './model/dx_opt.data')
    torch.save(dy_opt.state_dict(), './model/dy_opt.data')

Epoch 4/200, 0/1334, 0:00:00.034435
Epoch 4/200, 1/1334, 0:00:02.751266
Epoch 4/200, 2/1334, 0:00:03.356636
Epoch 4/200, 3/1334, 0:00:03.989207
Epoch 4/200, 4/1334, 0:00:04.586768
Epoch 4/200, 5/1334, 0:00:05.182239
Epoch 4/200, 6/1334, 0:00:05.781441
Epoch 4/200, 7/1334, 0:00:06.422473
Epoch 4/200, 8/1334, 0:00:06.999970
Epoch 4/200, 9/1334, 0:00:07.617385
Epoch 4/200, 10/1334, 0:00:08.208896
Epoch 4/200, 11/1334, 0:00:08.797780
Epoch 4/200, 12/1334, 0:00:09.432392
Epoch 4/200, 13/1334, 0:00:10.020016
Epoch 4/200, 14/1334, 0:00:10.601919
Epoch 4/200, 15/1334, 0:00:11.203396
Epoch 4/200, 16/1334, 0:00:11.823683
Epoch 4/200, 17/1334, 0:00:12.406706
Epoch 4/200, 18/1334, 0:00:12.987338
Epoch 4/200, 19/1334, 0:00:13.591880
Epoch 4/200, 20/1334, 0:00:14.190864
Epoch 4/200, 21/1334, 0:00:14.780128
Epoch 4/200, 22/1334, 0:00:15.379589
Epoch 4/200, 23/1334, 0:00:15.970984
Epoch 4/200, 24/1334, 0:00:16.571246
Epoch 4/200, 25/1334, 0:00:17.203488
Epoch 4/200, 26/1334, 0:00:17.787793
Epoch 4/200

Epoch 4/200, 219/1334, 0:02:13.576881
Epoch 4/200, 220/1334, 0:02:14.181736
Epoch 4/200, 221/1334, 0:02:14.802253
Epoch 4/200, 222/1334, 0:02:15.380167
Epoch 4/200, 223/1334, 0:02:15.987867
Epoch 4/200, 224/1334, 0:02:16.588105
Epoch 4/200, 225/1334, 0:02:17.217501
Epoch 4/200, 226/1334, 0:02:17.806287
Epoch 4/200, 227/1334, 0:02:18.387521
Epoch 4/200, 228/1334, 0:02:19.003320
Epoch 4/200, 229/1334, 0:02:19.600246
Epoch 4/200, 230/1334, 0:02:20.186296
Epoch 4/200, 231/1334, 0:02:20.793529
Epoch 4/200, 232/1334, 0:02:21.388341
Epoch 4/200, 233/1334, 0:02:21.997507
Epoch 4/200, 234/1334, 0:02:22.596660
Epoch 4/200, 235/1334, 0:02:23.218789
Epoch 4/200, 236/1334, 0:02:23.785418
Epoch 4/200, 237/1334, 0:02:24.432328
Epoch 4/200, 238/1334, 0:02:25.022299
Epoch 4/200, 239/1334, 0:02:25.600535
Epoch 4/200, 240/1334, 0:02:26.210746
Epoch 4/200, 241/1334, 0:02:26.784422
Epoch 4/200, 242/1334, 0:02:27.416725
Epoch 4/200, 243/1334, 0:02:28.009596
Epoch 4/200, 244/1334, 0:02:28.597187
Epoch 4/200,

Epoch 4/200, 435/1334, 0:04:23.197578
Epoch 4/200, 436/1334, 0:04:23.807315
Epoch 4/200, 437/1334, 0:04:24.410719
Epoch 4/200, 438/1334, 0:04:25.004851
Epoch 4/200, 439/1334, 0:04:25.606054
Epoch 4/200, 440/1334, 0:04:26.209907
Epoch 4/200, 441/1334, 0:04:26.807120
Epoch 4/200, 442/1334, 0:04:27.418646
Epoch 4/200, 443/1334, 0:04:28.004529
Epoch 4/200, 444/1334, 0:04:28.592636
Epoch 4/200, 445/1334, 0:04:29.196816
Epoch 4/200, 446/1334, 0:04:29.788406
Epoch 4/200, 447/1334, 0:04:30.383186
Epoch 4/200, 448/1334, 0:04:30.984907
Epoch 4/200, 449/1334, 0:04:31.597626
Epoch 4/200, 450/1334, 0:04:32.187682
Epoch 4/200, 451/1334, 0:04:32.815223
Epoch 4/200, 452/1334, 0:04:33.379149
Epoch 4/200, 453/1334, 0:04:33.978489
Epoch 4/200, 454/1334, 0:04:34.582390
Epoch 4/200, 455/1334, 0:04:35.181718
Epoch 4/200, 456/1334, 0:04:35.795211
Epoch 4/200, 457/1334, 0:04:36.376781
Epoch 4/200, 458/1334, 0:04:36.980630
Epoch 4/200, 459/1334, 0:04:37.584273
Epoch 4/200, 460/1334, 0:04:38.184547
Epoch 4/200,

Epoch 4/200, 651/1334, 0:06:32.796208
Epoch 4/200, 652/1334, 0:06:33.407416
Epoch 4/200, 653/1334, 0:06:33.986507
Epoch 4/200, 654/1334, 0:06:34.587194
Epoch 4/200, 655/1334, 0:06:35.199850
Epoch 4/200, 656/1334, 0:06:35.809541
Epoch 4/200, 657/1334, 0:06:36.392737
Epoch 4/200, 658/1334, 0:06:37.004626
Epoch 4/200, 659/1334, 0:06:37.612440
Epoch 4/200, 660/1334, 0:06:38.231938
Epoch 4/200, 661/1334, 0:06:38.825425
Epoch 4/200, 662/1334, 0:06:39.392200
Epoch 4/200, 663/1334, 0:06:40.031985
Epoch 4/200, 664/1334, 0:06:40.615091
Epoch 4/200, 665/1334, 0:06:41.187524
Epoch 4/200, 666/1334, 0:06:41.791864
Epoch 4/200, 667/1334, 0:06:42.386058
Epoch 4/200, 668/1334, 0:06:42.987681
Epoch 4/200, 669/1334, 0:06:43.590940
Epoch 4/200, 670/1334, 0:06:44.218336
Epoch 4/200, 671/1334, 0:06:44.822248
Epoch 4/200, 672/1334, 0:06:45.401943
Epoch 4/200, 673/1334, 0:06:46.002133
Epoch 4/200, 674/1334, 0:06:46.586138
Epoch 4/200, 675/1334, 0:06:47.198236
Epoch 4/200, 676/1334, 0:06:47.780685
Epoch 4/200,

Epoch 4/200, 867/1334, 0:08:42.376351
Epoch 4/200, 868/1334, 0:08:42.997177
Epoch 4/200, 869/1334, 0:08:43.603167
Epoch 4/200, 870/1334, 0:08:44.199531
Epoch 4/200, 871/1334, 0:08:44.804417
Epoch 4/200, 872/1334, 0:08:45.400973
Epoch 4/200, 873/1334, 0:08:45.987799
Epoch 4/200, 874/1334, 0:08:46.594261
Epoch 4/200, 875/1334, 0:08:47.204346
Epoch 4/200, 876/1334, 0:08:47.809375
Epoch 4/200, 877/1334, 0:08:48.380618
Epoch 4/200, 878/1334, 0:08:48.979973
Epoch 4/200, 879/1334, 0:08:49.598947
Epoch 4/200, 880/1334, 0:08:50.186128
Epoch 4/200, 881/1334, 0:08:50.789137
Epoch 4/200, 882/1334, 0:08:51.390846
Epoch 4/200, 883/1334, 0:08:51.991127
Epoch 4/200, 884/1334, 0:08:52.584948
Epoch 4/200, 885/1334, 0:08:53.201959
Epoch 4/200, 886/1334, 0:08:53.812156
Epoch 4/200, 887/1334, 0:08:54.390020
Epoch 4/200, 888/1334, 0:08:55.001843
Epoch 4/200, 889/1334, 0:08:55.609665
Epoch 4/200, 890/1334, 0:08:56.202332
Epoch 4/200, 891/1334, 0:08:56.792699
Epoch 4/200, 892/1334, 0:08:57.406235
Epoch 4/200,

...

Epoch 20/200, 896/1334, 3:38:11.773729
Epoch 20/200, 897/1334, 3:38:12.373464
Epoch 20/200, 898/1334, 3:38:12.963252
Epoch 20/200, 899/1334, 3:38:13.572788
Epoch 20/200, 900/1334, 3:38:14.153870
Epoch 20/200, 901/1334, 3:38:14.699590
Epoch 20/200, 902/1334, 3:38:15.276960
Epoch 20/200, 903/1334, 3:38:15.910198
Epoch 20/200, 904/1334, 3:38:16.479057
Epoch 20/200, 905/1334, 3:38:17.105604
Epoch 20/200, 906/1334, 3:38:17.695402
Epoch 20/200, 907/1334, 3:38:18.304309
Epoch 20/200, 908/1334, 3:38:18.889260
Epoch 20/200, 909/1334, 3:38:19.478921
Epoch 20/200, 910/1334, 3:38:20.105896
Epoch 20/200, 911/1334, 3:38:20.687531
Epoch 20/200, 912/1334, 3:38:21.285003
Epoch 20/200, 913/1334, 3:38:21.914511
Epoch 20/200, 914/1334, 3:38:22.491557
Epoch 20/200, 915/1334, 3:38:23.067470
Epoch 20/200, 916/1334, 3:38:23.697828
Epoch 20/200, 917/1334, 3:38:24.301484
Epoch 20/200, 918/1334, 3:38:24.816139
Epoch 20/200, 919/1334, 3:38:25.349632
Epoch 20/200, 920/1334, 3:38:25.887166
Epoch 20/200, 921/1334, 3

Epoch 20/200, 1104/1334, 3:40:13.914732
Epoch 20/200, 1105/1334, 3:40:14.497240
Epoch 20/200, 1106/1334, 3:40:15.106429
Epoch 20/200, 1107/1334, 3:40:15.693867
Epoch 20/200, 1108/1334, 3:40:16.299402
Epoch 20/200, 1109/1334, 3:40:16.892952
Epoch 20/200, 1110/1334, 3:40:17.514221
Epoch 20/200, 1111/1334, 3:40:18.126251
Epoch 20/200, 1112/1334, 3:40:18.711668
Epoch 20/200, 1113/1334, 3:40:19.308950
Epoch 20/200, 1114/1334, 3:40:19.888831
Epoch 20/200, 1115/1334, 3:40:20.507263
Epoch 20/200, 1116/1334, 3:40:21.106841
Epoch 20/200, 1117/1334, 3:40:21.649077
Epoch 20/200, 1118/1334, 3:40:22.171754
Epoch 20/200, 1119/1334, 3:40:22.738756
Epoch 20/200, 1120/1334, 3:40:23.282983
Epoch 20/200, 1121/1334, 3:40:23.883064
Epoch 20/200, 1122/1334, 3:40:24.452369
Epoch 20/200, 1123/1334, 3:40:24.992458
Epoch 20/200, 1124/1334, 3:40:25.578867
Epoch 20/200, 1125/1334, 3:40:26.201641
Epoch 20/200, 1126/1334, 3:40:26.783437
Epoch 20/200, 1127/1334, 3:40:27.402247
Epoch 20/200, 1128/1334, 3:40:28.005953


Epoch 20/200, 1309/1334, 3:42:14.461715
Epoch 20/200, 1310/1334, 3:42:14.976851
Epoch 20/200, 1311/1334, 3:42:15.468834
Epoch 20/200, 1312/1334, 3:42:16.088555
Epoch 20/200, 1313/1334, 3:42:16.673570
Epoch 20/200, 1314/1334, 3:42:17.283182
Epoch 20/200, 1315/1334, 3:42:17.837640
Epoch 20/200, 1316/1334, 3:42:18.372184
Epoch 20/200, 1317/1334, 3:42:18.893391
Epoch 20/200, 1318/1334, 3:42:19.462724
Epoch 20/200, 1319/1334, 3:42:20.087311
Epoch 20/200, 1320/1334, 3:42:20.679853
Epoch 20/200, 1321/1334, 3:42:21.237575
Epoch 20/200, 1322/1334, 3:42:21.775968
Epoch 20/200, 1323/1334, 3:42:22.404256
Epoch 20/200, 1324/1334, 3:42:23.006080
Epoch 20/200, 1325/1334, 3:42:23.598224
Epoch 20/200, 1326/1334, 3:42:24.208760
Epoch 20/200, 1327/1334, 3:42:24.744747
Epoch 20/200, 1328/1334, 3:42:25.301223
Epoch 20/200, 1329/1334, 3:42:25.890815
Epoch 20/200, 1330/1334, 3:42:26.508386
Epoch 20/200, 1331/1334, 3:42:27.098501
Epoch 20/200, 1332/1334, 3:42:27.700375
Epoch 20/200, 1333/1334, 3:42:28.295846


Epoch 21/200, 188/1334, 3:44:21.804087
Epoch 21/200, 189/1334, 3:44:22.411304
Epoch 21/200, 190/1334, 3:44:22.981502
Epoch 21/200, 191/1334, 3:44:23.611419
Epoch 21/200, 192/1334, 3:44:24.204421
Epoch 21/200, 193/1334, 3:44:24.805508
Epoch 21/200, 194/1334, 3:44:25.377828
Epoch 21/200, 195/1334, 3:44:25.996656
Epoch 21/200, 196/1334, 3:44:26.607811
Epoch 21/200, 197/1334, 3:44:27.191037
Epoch 21/200, 198/1334, 3:44:27.805468
Epoch 21/200, 199/1334, 3:44:28.320006
Epoch 21/200, 200/1334, 3:44:28.892102
Epoch 21/200, 201/1334, 3:44:29.520563
Epoch 21/200, 202/1334, 3:44:30.094297
Epoch 21/200, 203/1334, 3:44:30.719098
Epoch 21/200, 204/1334, 3:44:31.308409
Epoch 21/200, 205/1334, 3:44:31.913547
Epoch 21/200, 206/1334, 3:44:32.509136
Epoch 21/200, 207/1334, 3:44:33.117154
Epoch 21/200, 208/1334, 3:44:33.708821
Epoch 21/200, 209/1334, 3:44:34.321580
Epoch 21/200, 210/1334, 3:44:34.899808
Epoch 21/200, 211/1334, 3:44:35.507509
Epoch 21/200, 212/1334, 3:44:36.119268
Epoch 21/200, 213/1334, 3

Epoch 21/200, 399/1334, 3:46:27.779877
Epoch 21/200, 400/1334, 3:46:28.402173
Epoch 21/200, 401/1334, 3:46:28.982582
Epoch 21/200, 402/1334, 3:46:29.609632
Epoch 21/200, 403/1334, 3:46:30.191105
Epoch 21/200, 404/1334, 3:46:30.804260
Epoch 21/200, 405/1334, 3:46:31.397687
Epoch 21/200, 406/1334, 3:46:31.998298
Epoch 21/200, 407/1334, 3:46:32.596573
Epoch 21/200, 408/1334, 3:46:33.206830
Epoch 21/200, 409/1334, 3:46:33.811260
Epoch 21/200, 410/1334, 3:46:34.390749
Epoch 21/200, 411/1334, 3:46:35.007628
Epoch 21/200, 412/1334, 3:46:35.604654
Epoch 21/200, 413/1334, 3:46:36.204126
Epoch 21/200, 414/1334, 3:46:36.800088
Epoch 21/200, 415/1334, 3:46:37.403279
Epoch 21/200, 416/1334, 3:46:38.001108
Epoch 21/200, 417/1334, 3:46:38.601294
Epoch 21/200, 418/1334, 3:46:39.205293
Epoch 21/200, 419/1334, 3:46:39.794619
Epoch 21/200, 420/1334, 3:46:40.401250
Epoch 21/200, 421/1334, 3:46:41.000935
Epoch 21/200, 422/1334, 3:46:41.571087
Epoch 21/200, 423/1334, 3:46:42.210256
Epoch 21/200, 424/1334, 3

In [ ]:
close(logfile)